Preprocessing Menggunakan Spacy
===

**Modul untuk Pembelajaran Analitika Media Sosial**

**Author:** *Dr. Eng. Farrikh Alzami, M.Kom* , *Abu Salam, M.Kom*

Program Studi Sistem Informasi S1

***Universitas Dian Nuswantoro***

# Perbedaan NLTK dan Spacy

Spacy is better than NLTK in terms of performance.Here, there are some comparison

1- NLTK is a string processing library. It takes strings as input and returns strings or lists of strings as output.Whereas, spaCy uses object-oriented approach.When we parse a text, spaCy returns document object whose words and sentences are objects themselves.

2- spaCy has support for word vectors whereas NLTK does not.

3- In word tokenization and POS-tagging spaCy performs better, but in sentence tokenization, NLTK outperforms spaCy.

4- NLTK supports various languages whereas spaCy have statistical models for 7 languages (English, German, Spanish, French, Portuguese, Italian, and Dutch). It also supports named entities for multi language.

## memakai NLTK

In [1]:
# token
import nltk
from nltk.tokenize import word_tokenize

In [2]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [4]:
nltk.download('punkt_tab')
teks_nltk = word_tokenize_wrapper('Yahya beserta teman-teman TK suka melihat lumba-lumba di Batang Dolphin Center')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
teks_nltk

['Yahya',
 'beserta',
 'teman-teman',
 'TK',
 'suka',
 'melihat',
 'lumba-lumba',
 'di',
 'Batang',
 'Dolphin',
 'Center']

## memakai spacy

In [6]:
from spacy.lang.id import Indonesian
import spacy

nlp = Indonesian()  # use directly
nlp = spacy.blank('id')  # blank instance'
Teks = nlp('Yahya beserta teman-teman TK suka melihat lumba-lumba di Batang Dolphin Center')

In [7]:
Token_kata = [token.text for token in Teks]

In [8]:
Token_kata

['Yahya',
 'beserta',
 'teman-teman',
 'TK',
 'suka',
 'melihat',
 'lumba-lumba',
 'di',
 'Batang',
 'Dolphin',
 'Center']

# load library

In [9]:
import re
import string
import time
from copy import deepcopy

# load dataset

dataset ini menggunakan sentimen dan emosi.

kali ini kita akan menggunakan sentimen terlebih dahulu

silakan drop kolom Emosi

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('Dataset_Sentimen_Emosi.csv')

In [12]:
df.head()

,Tweet,Sentimen,Emosi
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,1
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,-1
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,1
3,Covid belum nyampe prigen mbak hmm hoax,0.0,-2
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,-2


In [13]:
df = df.drop(['Emosi'], axis=1)

In [14]:
df

,Tweet,Sentimen
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0
3,Covid belum nyampe prigen mbak hmm hoax,0.0
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0
...,...,...
899,Seluruh negara di dunia mengalami masa sulit k...,1.0
900,"Setelah covid dan skripsi disaster selesai, ma...",1.0
901,"Malam ini!! Projek ""BENDA BOLEH BINCANG"" 9 mal...",0.0
902,Pontang - panting di koyak covid 19,-1.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     904 non-null    object 
 1   Sentimen  903 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.3+ KB


# Preprocessing dataset
kita akan pakai modul dari AMS 01-03

disamping itu kita akan memakai modul  https://github.com/cbaziotis/ekphrasis

untuk instalasi, silakan buka tanda pagar dan jalankan

In [18]:
!pip install ekphrasis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00


In [19]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


/usr/local/lib/python3.11/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.11/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [20]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [21]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('rt'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text)

In [22]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text)

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

## lakukan pembersihan dengan memanggil fungsi yang didefinisikan diatas

In [23]:
i = 0
final_string = []
s = ""
for text in df['Tweet'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = lower(text)
    proc = change_stripe(text)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    #proc = remove_rt(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc)
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [24]:
df["step01"] = final_string

In [25]:
df.head(10)

,Tweet,Sentimen,step01
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,cegah mata rantai covid mari kita dirumah sa...
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,aku mohon yaallah semoga wabah covid menghil...
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,pemprov papua naikkan status jadi tanggap daru...
3,Covid belum nyampe prigen mbak hmm hoax,0.0,covid belum nyampe prigen mbak hmm hoax
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,nyuruh orang pintar lu aja togog itu kerumunan...
5,Pikir2 balik byk mnde plk nk setelkn lepas covid.,0.0,pikir balik byk mnde plk nk setelkn lepas covid
6,"Selamat pagi, hari jum'at. Jum'at keempat di k...",1.0,selamat pagi hari jumat jumat keempat di kala ...
7,"Hikmah di balik musibah covid-19, smg para pej...",1.0,hikmah di balik musibah covid smg para pejab...
8,Cegah covid-19 beserta jajaran Polsek Kuranji ...,1.0,cegah covid beserta jajaran polsek kuranji m...
9,Ya Allah kami memohon pada mu perkenankanlah d...,1.0,ya allah kami memohon pada mu perkenankanlah d...


## hapus data kosong

kadang ada data yang kosong. dan ini tidak bisa kita apa-apakan, maka dari itu kita hapus saja

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     904 non-null    object 
 1   Sentimen  903 non-null    float64
 2   step01    904 non-null    object 
dtypes: float64(1), object(2)
memory usage: 21.3+ KB


In [27]:
df_hapus = df[~df['step01'].str.contains(" ")]

In [28]:
df_hapus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 78 to 78
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     1 non-null      object 
 1   Sentimen  1 non-null      float64
 2   step01    1 non-null      object 
dtypes: float64(1), object(2)
memory usage: 32.0+ bytes


In [29]:
df_hapus.head(10)

,Tweet,Sentimen,step01
78,covid,0.0,covid


In [30]:
df_new = df[~df.isin(df_hapus)].dropna()

In [31]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 902 entries, 0 to 903
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     902 non-null    object 
 1   Sentimen  902 non-null    float64
 2   step01    902 non-null    object 
dtypes: float64(1), object(2)
memory usage: 28.2+ KB


In [32]:
df_new

,Tweet,Sentimen,step01
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,cegah mata rantai covid mari kita dirumah sa...
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,aku mohon yaallah semoga wabah covid menghil...
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,pemprov papua naikkan status jadi tanggap daru...
3,Covid belum nyampe prigen mbak hmm hoax,0.0,covid belum nyampe prigen mbak hmm hoax
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,nyuruh orang pintar lu aja togog itu kerumunan...
...,...,...,...
899,Seluruh negara di dunia mengalami masa sulit k...,1.0,seluruh negara di dunia mengalami masa sulit k...
900,"Setelah covid dan skripsi disaster selesai, ma...",1.0,setelah covid dan skripsi disaster selesai mau...
901,"Malam ini!! Projek ""BENDA BOLEH BINCANG"" 9 mal...",0.0,malam ini projek benda boleh bincang malam d...
902,Pontang - panting di koyak covid 19,-1.0,pontang panting di koyak covid


## normalisasi kata slang menjadi kata baku

In [33]:
# token
import nltk
from nltk.tokenize import word_tokenize

In [34]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [35]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [36]:
df_new.head(10)

,Tweet,Sentimen,step01,tokens
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,cegah mata rantai covid mari kita dirumah sa...,"[cegah, mata, rantai, covid, mari, kita, dirum..."
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,aku mohon yaallah semoga wabah covid menghil...,"[aku, mohon, yaallah, semoga, wabah, covid, me..."
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,pemprov papua naikkan status jadi tanggap daru...,"[pemprov, papua, naikkan, status, jadi, tangga..."
3,Covid belum nyampe prigen mbak hmm hoax,0.0,covid belum nyampe prigen mbak hmm hoax,"[covid, belum, nyampe, prigen, mbak, hmm, hoax]"
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,nyuruh orang pintar lu aja togog itu kerumunan...,"[nyuruh, orang, pintar, lu, aja, togog, itu, k..."
5,Pikir2 balik byk mnde plk nk setelkn lepas covid.,0.0,pikir balik byk mnde plk nk setelkn lepas covid,"[pikir, balik, byk, mnde, plk, nk, setelkn, le..."
6,"Selamat pagi, hari jum'at. Jum'at keempat di k...",1.0,selamat pagi hari jumat jumat keempat di kala ...,"[selamat, pagi, hari, jumat, jumat, keempat, d..."
7,"Hikmah di balik musibah covid-19, smg para pej...",1.0,hikmah di balik musibah covid smg para pejab...,"[hikmah, di, balik, musibah, covid, smg, para,..."
8,Cegah covid-19 beserta jajaran Polsek Kuranji ...,1.0,cegah covid beserta jajaran polsek kuranji m...,"[cegah, covid, beserta, jajaran, polsek, kuran..."
9,Ya Allah kami memohon pada mu perkenankanlah d...,1.0,ya allah kami memohon pada mu perkenankanlah d...,"[ya, allah, kami, memohon, pada, mu, perkenank..."


In [38]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [39]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

<ipython-input-39-e61659fbadf0>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
<ipython-input-39-e61659fbadf0>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [40]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [41]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [42]:
df_new["step02"] = final_string_tokens

In [43]:
df_new.head(10)

,Tweet,Sentimen,step01,tokens,final_tokens,step02
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,cegah mata rantai covid mari kita dirumah sa...,"[cegah, mata, rantai, covid, mari, kita, dirum...","[cegah, mata, rantai, covid, mari, kita, dirum...",cegah mata rantai covid mari kita dirumah saja...
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,aku mohon yaallah semoga wabah covid menghil...,"[aku, mohon, yaallah, semoga, wabah, covid, me...","[aku, mohon, yaallah, semoga, wabah, covid, me...",aku mohon yaallah semoga wabah covid menghilan...
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,pemprov papua naikkan status jadi tanggap daru...,"[pemprov, papua, naikkan, status, jadi, tangga...","[pemprov, papua, naikkan, status, jadi, tangga...",pemprov papua naikkan status jadi tanggap daru...
3,Covid belum nyampe prigen mbak hmm hoax,0.0,covid belum nyampe prigen mbak hmm hoax,"[covid, belum, nyampe, prigen, mbak, hmm, hoax]","[covid, belum, nyampe, prigen, mbak, hmm, hoax]",covid belum nyampe prigen mbak hmm hoax
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,nyuruh orang pintar lu aja togog itu kerumunan...,"[nyuruh, orang, pintar, lu, aja, togog, itu, k...","[nyuruh, orang, pintar, lu, aja, togog, itu, k...",nyuruh orang pintar lu aja togog itu kerumunan...
5,Pikir2 balik byk mnde plk nk setelkn lepas covid.,0.0,pikir balik byk mnde plk nk setelkn lepas covid,"[pikir, balik, byk, mnde, plk, nk, setelkn, le...","[pikir, balik, byk, mnde, plk, nk, setelkn, le...",pikir balik byk mnde plk nk setelkn lepas covid
6,"Selamat pagi, hari jum'at. Jum'at keempat di k...",1.0,selamat pagi hari jumat jumat keempat di kala ...,"[selamat, pagi, hari, jumat, jumat, keempat, d...","[selamat, pagi, hari, jumat, jumat, keempat, d...",selamat pagi hari jumat jumat keempat di kala ...
7,"Hikmah di balik musibah covid-19, smg para pej...",1.0,hikmah di balik musibah covid smg para pejab...,"[hikmah, di, balik, musibah, covid, smg, para,...","[hikmah, di, balik, musibah, covid, smg, para,...",hikmah di balik musibah covid smg para pejabat...
8,Cegah covid-19 beserta jajaran Polsek Kuranji ...,1.0,cegah covid beserta jajaran polsek kuranji m...,"[cegah, covid, beserta, jajaran, polsek, kuran...","[cegah, covid, beserta, jajaran, polsek, kuran...",cegah covid beserta jajaran polsek kuranji mel...
9,Ya Allah kami memohon pada mu perkenankanlah d...,1.0,ya allah kami memohon pada mu perkenankanlah d...,"[ya, allah, kami, memohon, pada, mu, perkenank...","[ya, allah, kami, memohon, pada, mu, perkenank...",ya allah kami memohon pada mu perkenankanlah d...


# simpan

In [44]:
df.to_csv('clean_dataset2.csv',sep=";")